In [35]:
import pandas as pd
import requests
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import datetime

from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import MobileApplicationClient

In [36]:
import os
%load_ext dotenv
%dotenv
try:
    from requests_oauthlib import OAuth2
except ModuleNotFoundError:
    import sys
    import os

    # I need this because requests_oauth gets installed in a weird place on my system
    sys.path.append('/usr/local/lib/python3.6/site-packages')
    from requests_oauthlib import OAuth2

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [37]:
OAuth2

requests_oauthlib.oauth2_auth.OAuth2

In [38]:
# Bring in fitbit API access items from .env file

access_token = os.getenv("ACCESS_TOKEN")
client_secret = os.getenv("CLIENT_SECRET")
client_id = os.getenv("CLIENT_ID")
callback_url = os.getenv("CALLBACK_URL")
authorization_uri = os.getenv("AUTHORIZATION_URI")

In [39]:
# %load ~/anaconda3/lib/python3.6/site-packages/requests_oauthlib/oauth2_auth.py# %load 
from __future__ import unicode_literals
from oauthlib.oauth2 import WebApplicationClient, InsecureTransportError
from oauthlib.oauth2 import is_secure_transport
from requests.auth import AuthBase


class OAuth2(AuthBase):
    """Adds proof of authorization (OAuth2 token) to the request."""

    def __init__(self, client_id=None, client=None, token=None):
        """Construct a new OAuth 2 authorization object.

        :param client_id: Client id obtained during registration
        :param client: :class:`oauthlib.oauth2.Client` to be used. Default is
                       WebApplicationClient which is useful for any
                       hosted application but not mobile or desktop.
        :param token: Token dictionary, must include access_token
                      and token_type.
        """
        self._client = client or WebApplicationClient(client_id, token=token)
        if token:
            for k, v in token.items():
                setattr(self._client, k, v)

    def __call__(self, r):
        """Append an OAuth 2 token to the request.

        Note that currently HTTPS is required for all requests. There may be
        a token type that allows for plain HTTP in the future and then this
        should be updated to allow plain HTTP on a white list basis.
        """
        if not is_secure_transport(r.url):
            raise InsecureTransportError()
        r.url, r.headers, r.body = self._client.add_token(r.url,
                http_method=r.method, body=r.body, headers=r.headers)
        return r

In [40]:
# Setup OAuth2 access to FitBit API

scope = ["activity", "heartrate", "location", "nutrition", "profile", "settings", "sleep", "social", "weight"]

client = MobileApplicationClient(client_id)

token = {
    'access_token': access_token,
    'token_type': 'Bearer'
}

fitbit = OAuth2Session(client_id, client=client, scope=scope, token=token)

{'user': {'age': 25,
  'ambassador': False,
  'autoStrideEnabled': True,
  'avatar': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_100_square.jpg',
  'avatar150': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_150_square.jpg',
  'avatar640': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_640_square.jpg',
  'averageDailySteps': 7822,
  'clockTimeDisplayFormat': '12hour',
  'corporate': False,
  'corporateAdmin': False,
  'country': 'US',
  'dateOfBirth': '1992-12-11',
  'displayName': 'Brenden R.',
  'displayNameSetting': 'name',
  'distanceUnit': 'en_US',
  'encodedId': '2F5M36',
  'familyGuidanceEnabled': False,
  'features': {'exerciseGoal': True},
  'firstName': 'Brenden',
  'foodsLocale': 'en_US',
  'fullName': 'Brenden Rossin',
  'gender': 'MALE',
  'glucoseUnit': 'en_US',
  'height': 180.4,
  'heightUnit': 'en_US',
  'isChild': False,
  'isGraduationAvailable': False,
  'l

In [41]:
# Create date list to loop through to get activity data

yesterday = datetime.datetime(2018,8,27)

date_list_end = ['2018-08-27']

for i in range(19):
    date = yesterday - datetime.timedelta(days = 31)
    date_list_end.append(date.strftime('%Y-%m-%d'))
    yesterday = date

In [42]:
date_list_end

['2018-08-27',
 '2018-07-27',
 '2018-06-26',
 '2018-05-26',
 '2018-04-25',
 '2018-03-25',
 '2018-02-22',
 '2018-01-22',
 '2017-12-22',
 '2017-11-21',
 '2017-10-21',
 '2017-09-20',
 '2017-08-20',
 '2017-07-20',
 '2017-06-19',
 '2017-05-19',
 '2017-04-18',
 '2017-03-18',
 '2017-02-15',
 '2017-01-15']

In [43]:
start_date_list = []

yesterday = datetime.datetime(2018,8,28)

for i in range(20):
    date = yesterday - datetime.timedelta(days = 31)
    start_date_list.append(date.strftime('%Y-%m-%d'))
    yesterday = date
    
start_date_list

['2018-07-28',
 '2018-06-27',
 '2018-05-27',
 '2018-04-26',
 '2018-03-26',
 '2018-02-23',
 '2018-01-23',
 '2017-12-23',
 '2017-11-22',
 '2017-10-22',
 '2017-09-21',
 '2017-08-21',
 '2017-07-21',
 '2017-06-20',
 '2017-05-20',
 '2017-04-19',
 '2017-03-19',
 '2017-02-16',
 '2017-01-16',
 '2016-12-16']

In [44]:
date_list = list(zip(start_date_list, date_list_end))

date_list

[('2018-07-28', '2018-08-27'),
 ('2018-06-27', '2018-07-27'),
 ('2018-05-27', '2018-06-26'),
 ('2018-04-26', '2018-05-26'),
 ('2018-03-26', '2018-04-25'),
 ('2018-02-23', '2018-03-25'),
 ('2018-01-23', '2018-02-22'),
 ('2017-12-23', '2018-01-22'),
 ('2017-11-22', '2017-12-22'),
 ('2017-10-22', '2017-11-21'),
 ('2017-09-21', '2017-10-21'),
 ('2017-08-21', '2017-09-20'),
 ('2017-07-21', '2017-08-20'),
 ('2017-06-20', '2017-07-20'),
 ('2017-05-20', '2017-06-19'),
 ('2017-04-19', '2017-05-19'),
 ('2017-03-19', '2017-04-18'),
 ('2017-02-16', '2017-03-18'),
 ('2017-01-16', '2017-02-15'),
 ('2016-12-16', '2017-01-15')]

In [45]:
r = fitbit.get(f'https://api.fitbit.com/1/user/-/sleep/date/2018-07-28/2018-08-27.json')
response = r.json()

In [46]:
response

{'sleep': [{'awakeCount': 0,
   'awakeDuration': 0,
   'awakeningsCount': 9,
   'dateOfSleep': '2018-08-25',
   'duration': 40080000,
   'efficiency': 97,
   'endTime': '2018-08-25T11:31:00.000',
   'logId': 19334478450,
   'minuteData': [{'dateTime': '00:22:30', 'value': '2'},
    {'dateTime': '00:23:30', 'value': '1'},
    {'dateTime': '00:24:30', 'value': '1'},
    {'dateTime': '00:25:30', 'value': '1'},
    {'dateTime': '00:26:30', 'value': '1'},
    {'dateTime': '00:27:30', 'value': '1'},
    {'dateTime': '00:28:30', 'value': '1'},
    {'dateTime': '00:29:30', 'value': '1'},
    {'dateTime': '00:30:30', 'value': '1'},
    {'dateTime': '00:31:30', 'value': '1'},
    {'dateTime': '00:32:30', 'value': '1'},
    {'dateTime': '00:33:30', 'value': '1'},
    {'dateTime': '00:34:30', 'value': '1'},
    {'dateTime': '00:35:30', 'value': '1'},
    {'dateTime': '00:36:30', 'value': '1'},
    {'dateTime': '00:37:30', 'value': '1'},
    {'dateTime': '00:38:30', 'value': '1'},
    {'dateTime': 

In [47]:
response['sleep']

[{'awakeCount': 0,
  'awakeDuration': 0,
  'awakeningsCount': 9,
  'dateOfSleep': '2018-08-25',
  'duration': 40080000,
  'efficiency': 97,
  'endTime': '2018-08-25T11:31:00.000',
  'logId': 19334478450,
  'minuteData': [{'dateTime': '00:22:30', 'value': '2'},
   {'dateTime': '00:23:30', 'value': '1'},
   {'dateTime': '00:24:30', 'value': '1'},
   {'dateTime': '00:25:30', 'value': '1'},
   {'dateTime': '00:26:30', 'value': '1'},
   {'dateTime': '00:27:30', 'value': '1'},
   {'dateTime': '00:28:30', 'value': '1'},
   {'dateTime': '00:29:30', 'value': '1'},
   {'dateTime': '00:30:30', 'value': '1'},
   {'dateTime': '00:31:30', 'value': '1'},
   {'dateTime': '00:32:30', 'value': '1'},
   {'dateTime': '00:33:30', 'value': '1'},
   {'dateTime': '00:34:30', 'value': '1'},
   {'dateTime': '00:35:30', 'value': '1'},
   {'dateTime': '00:36:30', 'value': '1'},
   {'dateTime': '00:37:30', 'value': '1'},
   {'dateTime': '00:38:30', 'value': '1'},
   {'dateTime': '00:39:30', 'value': '1'},
   {'dat

In [49]:
df = pd.DataFrame(response['sleep'])

df

,awakeCount,awakeDuration,awakeningsCount,dateOfSleep,duration,efficiency,endTime,logId,minuteData,minutesAfterWakeup,minutesAsleep,minutesAwake,minutesToFallAsleep,restlessCount,restlessDuration,startTime,timeInBed
0,0,0,9,2018-08-25,40080000,97,2018-08-25T11:31:00.000,19334478450,"[{'dateTime': '00:22:30', 'value': '2'}, {'dat...",0,650,18,0,9,18,2018-08-25T00:22:30.000,668
1,0,0,6,2018-08-24,20160000,97,2018-08-24T07:07:00.000,19334478449,"[{'dateTime': '01:30:30', 'value': '2'}, {'dat...",0,326,10,0,6,10,2018-08-24T01:30:30.000,336
2,0,0,7,2018-08-23,28860000,98,2018-08-23T07:51:00.000,19280780178,"[{'dateTime': '23:49:30', 'value': '1'}, {'dat...",0,471,10,0,7,10,2018-08-22T23:49:30.000,481
3,0,0,7,2018-08-22,28200000,98,2018-08-22T07:44:00.000,19280780177,"[{'dateTime': '23:53:30', 'value': '1'}, {'dat...",1,459,10,0,7,11,2018-08-21T23:53:30.000,470
4,0,0,9,2018-08-17,28140000,96,2018-08-17T07:28:00.000,19254431983,"[{'dateTime': '23:39:00', 'value': '2'}, {'dat...",0,450,19,0,9,19,2018-08-16T23:39:00.000,469
5,0,0,10,2018-08-15,23400000,96,2018-08-15T07:43:00.000,19254431982,"[{'dateTime': '01:13:00', 'value': '1'}, {'dat...",4,369,17,0,10,21,2018-08-15T01:13:00.000,390
6,1,1,13,2018-08-11,35100000,95,2018-08-11T09:08:30.000,19163913786,"[{'dateTime': '23:23:00', 'value': '2'}, {'dat...",0,557,28,0,12,27,2018-08-10T23:23:00.000,585
7,0,0,6,2018-08-08,28320000,99,2018-08-08T07:51:30.000,19163913785,"[{'dateTime': '23:59:30', 'value': '2'}, {'dat...",0,465,7,0,6,7,2018-08-07T23:59:30.000,472
8,0,0,7,2018-08-07,27720000,98,2018-08-07T07:58:00.000,19163913784,"[{'dateTime': '00:15:30', 'value': '1'}, {'dat...",0,453,9,0,7,9,2018-08-07T00:15:30.000,462


In [50]:
len(response['sleep'])

9

In [34]:
# Get all sleep data in the last 20 months from my fitbit

# FIX ONE FIRST BEFORE TRYING ALL

# for index, value in enumerate(date_list):
#     start_date = value[0]
#     end_date = value[1]
#     r = fitbit.get(f'https://api.fitbit.com/1/user/-/sleep/date/{start_date}/{end_date}.json')
#     response = r.json()
#     sleep_json = response['activities-heart']
#     heartrate_df = pd.DataFrame(heartrate)
#     if index == 0:
#         all_heartrate_df = heartrate_df
#     else:
#         all_heartrate_df = pd.concat([heartrate_df, heartrate_df])

# all_heartrate_df.rename(index=str, columns={"dateTime": "Date", "value": "HeartRate"}, inplace=True)

# all_heartrate_df

In [ ]:
# fitbit_df_HR = all_HR_df.copy(deep=True)

In [132]:
# fitbit_df_HR.Date = pd.to_datetime(fitbit_df_HR.Date)
# fitbit_df_HR.HeartRate = fitbit_df_HR.HeartRate.astype('int64')

In [133]:
# fitbit_df_HR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620 entries, 0 to 619
Data columns (total 4 columns):
Date               620 non-null datetime64[ns]
Steps              620 non-null int64
Distance           620 non-null float64
Calories_Burned    620 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 24.2 KB


In [134]:
# fitbit_df_HR.head()

,Date,Steps,Distance,Calories_Burned
0,2018-07-28,4031,3.01921,2237
1,2018-07-29,0,0.00000,1851
2,2018-07-30,0,0.00000,1851
3,2018-07-31,3985,2.98476,2486
4,2018-08-01,4173,3.12557,2434


In [136]:
# len(fitbit_df_HR[fitbit_df_HR['HeartRate'] < 1])

51

In [137]:
# fitbit_df_HR.to_pickle("Data/fitbit_heart_rate.pkl")    